# 开始使用LangGraph将 LangChain 组件组合成功能齐全的应用程序。
# 构建聊天机器人

In [6]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [7]:
import getpass
import os

if not os.environ.get("DEEPSEEK_API_KEY"):
  os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter API key for DeepSeek: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

Enter API key for DeepSeek:  ········


In [47]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! Nice to meet you. 😊\n\nWhat can I help you with today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 9, 'total_tokens': 28, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'b39528a3-e04d-4003-b4f5-2df7e45ea099', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--14d4a5f6-75c3-4f1b-912e-d281b381a3e7-0', usage_metadata={'input_tokens': 9, 'output_tokens': 19, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [48]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='I don’t have access to your name unless you tell me. In our conversation, you haven’t shared it yet — so I’m afraid I don’t know! 😊  \nIf you’d like, you can tell me your name, and I’ll be happy to use it in our conversation.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 9, 'total_tokens': 75, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '32a3339a-ed91-465a-a7ac-065ba5f9bcec', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--1a5aee95-80ee-4cbe-a541-e28692b6a5f7-0', usage_metadata={'input_tokens': 9, 'output_tokens': 66, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

由此可以，上面两次调用是分开的，模型并没有记忆这两次调用

In [49]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! You introduced yourself at the beginning of our conversation.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 28, 'total_tokens': 43, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 28}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '4893f20a-ae9b-430a-b827-60b95cd21d76', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c866ce3a-a6a8-45fb-83db-264ff4dd2869-0', usage_metadata={'input_tokens': 28, 'output_tokens': 15, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

当给出上下文的时候，模型才能够记忆，并且给出正确答案

In [50]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [51]:
config = {"configurable": {"thread_id": "abc123"}}

In [52]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello Bob! Nice to meet you. 😊  
What can I help you with today?


In [53]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is **Bob**! You mentioned it at the very beginning. 😊


In [54]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t have access to your name unless you tell me. If we’ve chatted before, I don’t retain personal information between conversations for privacy reasons.  

Would you like to tell me your name? I’d be happy to use it in our conversation! 😊


In [55]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is **Bob**! 😊


In [56]:

# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I’m afraid I don’t have access to your name unless you tell me! 😊  
If you’d like, you can share your name with me, and I’ll be happy to use it in our conversation.


In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [19]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc345"}}
query = "你好哇，我是海盗王！"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

（拍腿大笑）哈哈！原来是大名鼎鼎的海盗王驾到！（举起木酒杯）为我们的相遇干杯！您这艘宝贝船最近在哪片海域发财啊？要不要听听我刚从酒馆听来的藏宝图消息？


In [23]:
query = "我是谁?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

（单眼一眯，咧嘴露出金牙）嗬！这不是明知故问嘛！您刚拍着胸脯说自个儿是威震七海的**海盗王**，转头就忘啦？（举起朗姆酒桶豪饮一口）要不就是...您想试试老水手记不记得每个好汉的名号？


In [24]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [25]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [26]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Chinese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好，Bob！很高兴认识你。有什么我可以帮助你的吗？


In [27]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你的名字是Bob。


# 由于我是用deepseek模型，并不支持openai的get_num_tokens_from_messages()方法，直接使用trim_messages 函数需要这个方法来计算消息的令牌数
# 所以需要自定义一个令牌计数函数

In [69]:
import tiktoken
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# 自定义令牌计数函数
def count_tokens(messages):
    """
    计算消息列表的令牌数，适用于 DeepSeek 模型
    """
    # DeepSeek 使用与 OpenAI 相同的令牌化器
    try:
        encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")  # 或 "gpt-4"
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")  # 回退到基础编码
    
    # 将 LangChain 消息转换为 OpenAI 格式
    formatted_messages = []
    for message in messages:
        if isinstance(message, SystemMessage):
            role = "system"
        elif isinstance(message, HumanMessage):
            role = "user"
        elif isinstance(message, AIMessage):
            role = "assistant"
        else:
            role = "user"  # 默认
        
        content = message.content
        formatted_messages.append({"role": role, "content": content})
    
    # 基于 OpenAI 的令牌计数逻辑
    tokens_per_message = 3  # 每条消息的开销
    tokens_per_name = 1
    num_tokens = 0
    
    for message in formatted_messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(str(value)))
            if key == "name":  # 如果有名字字段
                num_tokens += tokens_per_name
    
    num_tokens += 3  # 每次回复的开销
    return num_tokens

# 创建 trimmer，使用自定义令牌计数器
trimmer = trim_messages(
    max_tokens=105,  # 这个的token数需要按需调整，完全照搬教程的数字可能太大或者太小
    strategy="last",  # 保留最后的消息
    token_counter=count_tokens,  # 使用自定义函数
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# 测试消息列表
messages = [
    SystemMessage(content="你是一个能够记忆和回顾对话历史的助手"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

# 执行截断
result = trimmer.invoke(messages)
print("截断后的消息:")
for msg in result:
    print(f"- {msg.type}: {msg.content}")


截断后的消息:
- system: 你是一个能够记忆和回顾对话历史的助手
- human: hi! I'm bob
- ai: hi!
- human: I like vanilla ice cream
- ai: nice
- human: whats 2 + 2
- ai: 4
- human: thanks
- ai: no problem!
- human: having fun?
- ai: yes!


In [70]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    """
    # 这里用来调试，看看截断前后的保留消息的变化
    print("=== 截断前的消息 ===")
    for msg in state["messages"]:
        print(f"- {msg.type}: {msg.content}")
    
    trimmed_messages = trimmer.invoke(state["messages"])
    
    print("=== 截断后的消息 ===")
    for msg in trimmed_messages:
        print(f"- {msg.type}: {msg.content}")
    """
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [71]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to your name from our previous conversation. You haven't told me your name yet!


In [72]:
config = {"configurable": {"thread_id": "abc678"}}
query = "我问过哪些数学问题?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

根据我们的对话历史，你只问过一个数学问题：
“whats 2 + 2”

我回答的是“4”。


In [73]:
print(f"原始消息数量: {len(messages)}")
print(f"截断后消息数量: {len(result)}")
print(f"估计令牌数: {count_tokens(result)}")

# 查看具体哪些消息被保留
for i, msg in enumerate(result):
    print(f"{i+1}. {msg.type}: {msg.content}")


原始消息数量: 11
截断后消息数量: 11
估计令牌数: 99
1. system: 你是一个能够记忆和回顾对话历史的助手
2. human: hi! I'm bob
3. ai: hi!
4. human: I like vanilla ice cream
5. ai: nice
6. human: whats 2 + 2
7. ai: 4
8. human: thanks
9. ai: no problem!
10. human: having fun?
11. ai: yes!


In [74]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have access to your name from our previous conversation. You've only asked me about math problems so far. If you'd like me to know your name, please feel free to tell me!


In [75]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "Chinese"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|当然|可以|，|T|odd|！|这里|有一个|轻松|的小|笑话|：

|有一天|，|一只|企|鹅|走进|一家|酒吧|，|它|走到|吧|台|前|对|酒|保|说|：“|请问|你|见过|我|爸爸|吗|？”|  
|酒|保|摇摇头|说|：“|没有|啊|，|他|长|什么|样子|？”|  
|企|鹅|叹了口气|：“|唉|，|就是|戴着|黑|帽子|、|穿着|白|衬衫|和|黑|西装|的那个|！”|  
|酒|保|一愣|：“|等等|……|你说的|这不|就是你|吗|？”|  
|企|鹅|眨|眨眼|：“|对|呀|，|所以我|可能|迷|路了|！”

|希望|这个|笑话|能|让你|开心|一笑|！|😊||